In [9]:
# Notes on notebook: 
# Notebook assumes two playlists already in spotify account: 'Pull' which is Nasheeds (must be hand curated due to 
# no instruments/vocals only) and 'Ramadan Daily' which  will be used in script to place podcasts & nasheeds. 
# Currently supports multiple podcasts, but only all surahs from juz for each Qari. Recommended to setup method run
# automatically every day, otherwise run yourself every day for a 'fresh' playlist each day of Ramadan. Note that 
# if pulling surahs, the first juz the surah is in will pull the entire surah and leave it out of next day/juz. 

# ------------------ PACKAGE IMPORT --------------------#
# Packages used in script

import spotipy 
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import spotipy.util as util
# DOCS of package https://spotipy.readthedocs.io/en/2.22.1/#getting-started
# https://stackoverflow.com/questions/tagged/spotify?tab=newest&page=1&pagesize=15
# https://www.newline.co/@bchiang7/breaking-down-oauth-with-spotify--90463996

import os
# DOCS of package https://docs.python.org/3/library/os.html

from datetime import date
# DOCS of package https://docs.python.org/3/library/datetime.html

import time
from time import sleep
# DOCS of package https://docs.python.org/3/library/time.html

import re
# DOCS of package https://docs.python.org/3/library/re.html

import random
# DOCS of package https://docs.python.org/3/library/random.html

import requests
# DOCS of package https://pypi.org/project/requests/

# -------------- END OF PACKAGE IMPORT -----------------#

# ------------------ SPOTIFY KEYS ----------------------#
# Various assignments that are used in the spotify api call 

SPOTIPY_USER_NAME = 'xxx'
SPOTIPY_CLIENT_ID = 'xxx'
SPOTIPY_CLIENT_SECRET = 'xxx'
SPOTIPY_REDIRECT_URI = 'xxx'
LOG_FOLDER_PATH =r'xxx/xxx/xxx'

# --------------- END OF SPOTIFY KEYS ------------------#


In [10]:
# --------------- HELPER FUNCTIONS  --------------------#

def refresh():
    #  https://stackoverflow.com/questions/49239516/spotipy-refreshing-a-token-with-authorization-code-flow
    global token_info, sp
    # Get the global values of token_info and the spotipy object
    if sp_oauth.is_token_expired(token_info):
        # if the token is expired 
        token_info = sp_oauth.refresh_access_token(token_info['refresh_token'])
        # refresh the token
        token = token_info['access_token']
        sp = spotipy.Spotify(auth=token)
        # recreate spotipy object 

def episode_pull(show_id, 
                 number_of_epsiodes = 1,
                 length_min = 15, 
                 length_max = 30, 
                 random_choice = True, 
                 exclusion_words = [''], 
                 filter_for = [''], 
                 topic_filter = ['']):   
    '''
    Returns podast episodes from spotify api.

            Parameters:
                    show_id (str): ID (not uri) of show
                    number_of_epsiodes (int): number of desired episodes
                    length_min (int): min number of minutes for episode
                    length_max (int): max number of minutes for episode
                    random_choice (bool): whether or not to pull random episodes 
                        FALSE: uses word in topic filter to filter episodes 
                        TRUE: randomly picks from episodes in intervals
                    exclusion_words (list, str): list of words to exclude 
                    filter_for (list, str): list of words to filter episode names for
                    topic_filter (list, str): should be use to 'filter' episode selection

            Returns:
                    A log of episode names and ids place in folder specified 
    '''
    global LOG_FOLDER_PATH, sp
    length_min = length_min*60000 # converts ms to minutes
    length_max = length_max*60000
    time.sleep(5) # So we don't get an api-over requested error 
    refresh() # To refresh the token 
    podcast = sp.show_episodes(show_id) # grab the first pull of the podcast
    shows_to_add = [] # empty list to add the episodes
    next_load = 0 # For the offset podcast reload
    i = 0 # To continue the while loop
    while i < ((podcast['total'] + 50 - 1) // 50): # intervals of 50, due to limit in show_episodes
        for show in range(len(podcast['items'])):
            # for each episode in podcast
            if (podcast['items'][show]['duration_ms'] < length_max and 
                # if the duration is under length max
            podcast['items'][show]['duration_ms'] > length_min and 
                # if the duration is over length min
            podcast['items'][show]['explicit'] == False and 
                # if the episode isn't explicit
            podcast['items'][show]['is_playable'] == True and 
                # and if the episode is playable
            not any(words in podcast['items'][show]['name'].lower() for words in exclusion_words) and
                # and there are no words in the name that are in exclusion words
            any(words in podcast['items'][show]['name'].lower() for words in filter_for)):
                # looking for any words in filte for
                shows_to_add.append(re.sub(r'[^A-Za-z0-9 ]+', '', podcast['items'][show]['name']))
                # Strip the string of non-alphabeticnumerics and add the name of the podcast
                shows_to_add.append(podcast['items'][show]['release_date'] + ' -id- ' + podcast['items'][show]['uri'])
                # get the release date and uri and create a id string
        shows_to_add = list(shows_to_add)
        # make shows_to_add a list
        for name in list_of_files:
            # for each file in list_of_files
            if str(date.today()) not in name:
                # if the name of the file does not include todays date
                with open(name) as f:
                    # open the file
                    contents = f.read()
                    # read the contents of the file
                    contents = contents.split("||")
                    # split on the double ||
                    contents = list(contents)
                    # create a list
                    contents = [x.rstrip() for x in contents]
                    # strip end whitespace
                    contents = [x.strip(' ') for x in contents]
                    # strip spaces
                    f.close()
                    # close the file
                    shows_to_add = [elem for elem in shows_to_add if elem not in contents ]
                    # Checks previous day's files to make sure that we're not adding episodes that 
                    # were already added previously 

        next_load = next_load + 50 # For the offset 
        podcast = sp.show_episodes(show_id, offset = next_load) # Pulls the next load in 
        i = i + 1 # For the while loop

    if random_choice: 
        # if random choice is true
        multiple = int((len(shows_to_add)/number_of_epsiodes)) & ~1
        # Get intervals, but also round down to nearest even number (even number needed for index pull)
        start_value = 0
        # to go through all the episodes
        shows_to_add_helper = []
        i = 1
        # for while loop
        while i < number_of_epsiodes+1:
            # while the number of episodes is above i
            index = random.randrange(start_value, multiple*i, 2)
            # get a random range to index
            shows_to_add_helper.append((shows_to_add[index], shows_to_add[index+1]))
            # index the show
            i = i + 1
            # for while loop
            start_value = start_value + multiple
            # move on to the next interval of shows
        shows_to_add = shows_to_add_helper
    else:
        shows_to_add = [(name, episode_id) for name, episode_id in zip(shows_to_add, shows_to_add[1:]) if any(word in name for word in topic_filter)]
        # pull the name and id for shows that match the topic filter
        shows_to_add = random.sample(shows_to_add, number_of_epsiodes)
        # randomly samples this so that number of episodes is honored

    for show in range(len(shows_to_add)):
        # for each show in shows_to_add
        with open(LOG_FOLDER_PATH + '/items_added_to_playlist_' + str(date.today()) + '.txt', 'a') as f:
                                                f.write('\n' + "|| " + "\n||".join(shows_to_add[show]))
                                                time.sleep(1)
                                                f.close()
        # Adds the names of the shows and the ids to the log
        
# ------------- END OF HELPER FUNCTIONS ----------------#

# ------------------------ SETUP -----------------------#

os.chdir('xxx')
# Set working directory 

## ------- PATH TO FOLDER ---------- ##

list_of_files = []
# This will contain all the previously listen to 'log' files

for root, dirs, files in os.walk(LOG_FOLDER_PATH):
    for file in files:
        list_of_files.append(os.path.join(root,file))
        # This goes into the folder and grabs all the files present so that 
        # everything within the folder is accounted for (all text files)
    
## ------- ISLAMIC DATE ------------ ##  
# https://aladhan.com/islamic-calendar-api
# Get the hijri date for the Quran 30 for 30 podcast & 
# get surahs in Juz for Quran Arabic & English

hijri_date = requests.get("http://api.aladhan.com/v1/gToH/" + date.today().strftime("%d-%m-%Y")).json()['data']['hijri']['day']
hijri_date = str(int(hijri_date) + 1)
surahs_in_juz = requests.get("http://api.alquran.cloud/v1/juz/" + hijri_date + "/en.asad").json()
surahs_in_juz = list(surahs_in_juz['data']['surahs'].keys())

# ----------------- END OF SETUP -----------------------#


In [ ]:
# ----------------- API CONNECTION  --------------------#
# TOKEN issues still a little unclear, answers found here: 
# https://stackoverflow.com/questions/49239516/spotipy-refreshing-a-token-with-authorization-code-flow

## ------- SCOPE DEFINITION -------- ##
# Scopes defined for app, see all scopes here: 
# https://developer.spotify.com/documentation/general/guides/authorization/scopes/

scope = 'playlist-modify-private playlist-modify-public'
# Define the scope, which is to modify a private playlist
# and a public playlist

## ---- AUTHORIZATION & TOKEN ------ ##
# Token authorization and usage 

sp_oauth = oauth2.SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI,scope=scope)
# 'LOGIN' to the spotify API with keys above 
token_info = sp_oauth.get_cached_token() 
# Get the stored token (unsure where this is being stored and how?)
token = token_info['access_token']
# Call the token info with access_token
sp = spotipy.Spotify(auth=token)
# actualy login now with the token

# ------------ END OF API CONNECTION  ------------------#

# -------------- PLAYLIST SELECTIONS  ------------------#
# Select both my Ramadan Daily and Pull playlist 
# Ramadan daily will be cleared and refreshed daily, while 
# pull contains Islamic Nasheeds/songs that will be randomly 
# selected and added to Ramadan Daily 

## --------- RAMADAN DAILY  -------- ##

refresh()
user_playlists = sp.current_user_playlists() # Gets all the playlists in account
for user_playlists_id in range(len(user_playlists['items'])):
    # for each playlist in account
        if user_playlists['items'][user_playlists_id]['name'] == 'Ramadan Daily':
            # check and see if the name is Ramadan Daily
            ramadan_daily = user_playlists['items'][user_playlists_id]['id'] 
            # if it is, pull the ID of the playlist and assign to ramadan_daily variable

refresh()
response = sp.playlist_items(ramadan_daily, limit=100)
# gets all the items in Ramadan Daily. Limit is currently 100 items, if 
# adding more than 100 items to playlist on average please adjust
items_in_playlist = []
# to store items in playlist
for items in range(len(response['items'])):
    # for each item in playlist
    items_in_playlist.append(response['items'][items]['track']['uri'])
    # pull the item uri and add to items in playlist
sp.playlist_remove_all_occurrences_of_items(ramadan_daily,items_in_playlist)
# delets all items in playlist from previous day

## --------- PULL PLAYLIST  -------- ##

for user_playlists_id in range(len(user_playlists['items'])):
        if user_playlists['items'][user_playlists_id]['name'] == 'Pull':
            pull_songs = user_playlists['items'][user_playlists_id]['id']  
pull_songs = sp.playlist(pull_songs)['tracks']
songs_in_pull = []
for songs in range(len(pull_songs['items'])):
    songs_in_pull.append(pull_songs['items'][songs]['track']['id'])
random.shuffle(songs_in_pull)


# ----------- END OF PLAYLIST SELECTIONS  --------------#

# ------------- ISLAMIC SPEAKERS/PODCASTS --------------#

## ---------- HAMZA YUSUF ---------- ##
# https://open.spotify.com/show/6ab78O9ddBnafNaLeLpuu2

try:
    episode_pull(show_id = '6ab78O9ddBnafNaLeLpuu2', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 60, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## ---------- YASIR QADHI ---------- ##
# https://open.spotify.com/show/5ZtWvDlzUXGneVr63EToc3

try:
    episode_pull(show_id = '5ZtWvDlzUXGneVr63EToc3', 
             number_of_epsiodes = 1,
             length_min = 40, 
             length_max = 60, 
             random_choice = True, 
             exclusion_words = ['part', 'PT', 'Pt', 'pt', 'Ask Shaykh'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## --------- OMAR SULEIMAN --------- ##
# https://open.spotify.com/show/3dLmv3iMDVU0RxeVoxkQkV

try:
    episode_pull(show_id = '3dLmv3iMDVU0RxeVoxkQkV', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 30, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', '30 for 30', 'first', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## ----------- DOUBLETAKE ---------- ##
# https://open.spotify.com/show/4wGVPBqTEv6ojsTn6vrjOv

try:
    episode_pull(show_id = '4wGVPBqTEv6ojsTn6vrjOv', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 60, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## ----------- THE FIRSTS --------- ##
# https://open.spotify.com/show/49rfo5mYgXsvGRNnp0rkqT

try:
    episode_pull(show_id = '49rfo5mYgXsvGRNnp0rkqT', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 60, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## -------- QURAN 30 FOR 30 ------- ##
# https://open.spotify.com/show/5rjpsAFtpKcJHZo3jISA6Q

try:
    episode_pull(show_id = '5rjpsAFtpKcJHZo3jISA6Q', 
             number_of_epsiodes = 3,
             length_min = 15, 
             length_max = 85, 
             random_choice = False, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = ['30 for 30'], 
             topic_filter = ['Juz ' + hijri_date + ' '])
except:
    pass
    
## -------- YASMIN MOGAHED -------- ##
# https://open.spotify.com/show/3KbF311SaEo1DVMzvsz2Qw
try:
    episode_pull(show_id = '3KbF311SaEo1DVMzvsz2Qw', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 30, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## ---------- ALI HAMMUDA --------- ##
# https://open.spotify.com/show/7G0LyuwuOrYVJsVjBs9N6h

try:
    episode_pull(show_id = '7G0LyuwuOrYVJsVjBs9N6h', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 30, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## -------- HAIFAA YOUNIS --------- ##
# https://open.spotify.com/show/2ByB16ix13zaJHwRgfMMQE

try:
    episode_pull(show_id = '2ByB16ix13zaJHwRgfMMQE', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 30, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## ---------- BILAL ASSAD --------- ##
# https://open.spotify.com/show/7pKaVoswCSj2w4elbWitVO

try:
    episode_pull(show_id = '7pKaVoswCSj2w4elbWitVO', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 30, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

## ---------- ABU BAKR ZOUD ------- ##
# https://open.spotify.com/show/2iYidyn1TyjczINcXXwC4R

try:
    episode_pull(show_id = '2iYidyn1TyjczINcXXwC4R', 
             number_of_epsiodes = 1,
             length_min = 15, 
             length_max = 30, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [''], 
             topic_filter = [''])
except:
    pass

# --------- END OF ISLAMIC SPEAKERS/PODCASTS -----------#

# -------------- QURAN [ENGLISH & ARABIC] --------------#

## --- NOREEN MUHAMMAD SIDDIQUE --- ##
# https://open.spotify.com/show/7n7rxGIqUFbhNAD6NwIW0f

try:
    episode_pull(show_id = '7n7rxGIqUFbhNAD6NwIW0f', 
             number_of_epsiodes = len(surahs_in_juz),
                 # This pulls the exact number of surahs per juz
             length_min = 0.5, 
             length_max = 180, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = [str(surahs).zfill(3) for surahs in surahs_in_juz], 
                 # This filters for the specific Juz, podcast have 3 numbers 
                 # for the surah name -- 001 | 002... Which is why filter for 
                 # is a list comp with zfill 3
             topic_filter = [''])
except:
    pass

## ------- QURAN IN ENGLISH ------- ##
# https://open.spotify.com/show/1llBPascXiLSvgbzPJSK34

try:
    episode_pull(show_id = '1llBPascXiLSvgbzPJSK34', 
             number_of_epsiodes = len(surahs_in_juz),
                 # This pulls the exact number of surahs per juz
             length_min = 0.5, 
             length_max = 180, 
             random_choice = True, 
             exclusion_words = ['PT', 'Pt', 'pt', 'part'], 
             filter_for = ['{0} '.format(surahs) for surahs in surahs_in_juz], 
                 # This filters for the specific Juz, podcast has space after 
                 # surah name, which is why filter for is a list comp with space
             topic_filter = [''])
except:
    pass

# ---------- END OF QURAN [ENGLISH & ARABIC] -----------#

# ------------------- PODCAST EPISODES -----------------#

list_of_files = []
# This will contain all the previously listen to 'log' files

for root, dirs, files in os.walk(LOG_FOLDER_PATH):
    for file in files:
        list_of_files.append(os.path.join(root,file))

for name in list_of_files:
        if str(date.today()) in name:
            with open(name) as f:
                contents = f.read()
                contents = contents.split("||")
                contents = list(contents)
                contents = [x.rstrip() for x in contents]
                contents = [x.strip(' ') for x in contents]

contents = [podcast_episode for podcast_episode in contents if "-id-" in podcast_episode]
contents = [podcast_episode.split(" ")[-1] for podcast_episode in contents]
refresh()
total_time = 0
for episode in range(len(contents)):
    try:
        total_time = total_time + round(sp.episode(contents[episode])['duration_ms']/60000)
    except: 
        pass 
total_time = total_time/60
print(total_time)
random.shuffle(contents)

# -------------- END OF PODCAST EPISODES ---------------#

add_to_playlist = [(podcast_epsiode, song_one, song_two) for podcast_epsiode, song_one, song_two in zip(contents, songs_in_pull[0::2],songs_in_pull[1::2])]
add_to_playlist = [item for sublist in [list(tup) for tup in add_to_playlist] for item in sublist]

refresh()
sp.playlist_add_items(ramadan_daily, add_to_playlist, position=None)
